In [1]:
import pickle
import time
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

from alexnet import AlexNet

nb_classes = 43
epochs = 10
batch_size = 128

with open('./train.p', 'rb') as f:
    data = pickle.load(f)

X_train, X_val, y_train, y_val = train_test_split(data['features'], data['labels'], test_size=0.33, random_state=0)

print("Extracting data:")
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

features = tf.placeholder(tf.float32, (None, 32, 32, 3))
labels = tf.placeholder(tf.int64, None)
resized = tf.image.resize_images(features, (227, 227))

# Returns the second final layer of the AlexNet model,
# this allows us to redo the last layer for the traffic signs
# model.
fc7 = AlexNet(resized, feature_extract=True)
fc7 = tf.stop_gradient(fc7)
shape = (fc7.get_shape().as_list()[-1], nb_classes)
fc8W = tf.Variable(tf.truncated_normal(shape, stddev=1e-2))
fc8b = tf.Variable(tf.zeros(nb_classes))
logits = tf.nn.xw_plus_b(fc7, fc8W, fc8b)

cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels)
loss_op = tf.reduce_mean(cross_entropy)
opt = tf.train.AdamOptimizer()
train_op = opt.minimize(loss_op, var_list=[fc8W, fc8b])
init_op = tf.global_variables_initializer()

preds = tf.arg_max(logits, 1)
accuracy_op = tf.reduce_mean(tf.cast(tf.equal(preds, labels), tf.float32))


def eval_on_data(X, y, sess):
    total_acc = 0
    total_loss = 0
    for offset in range(0, X.shape[0], batch_size):
        end = offset + batch_size
        X_batch = X[offset:end]
        y_batch = y[offset:end]

        loss, acc = sess.run([loss_op, accuracy_op], feed_dict={features: X_batch, labels: y_batch})
        total_loss += (loss * X_batch.shape[0])
        total_acc += (acc * X_batch.shape[0])

    return total_loss/X.shape[0], total_acc/X.shape[0]

print("Starting training...")

with tf.Session() as sess:
    sess.run(init_op)

    for i in range(epochs):
        # training
        X_train, y_train = shuffle(X_train, y_train)
        t0 = time.time()
        for offset in range(0, X_train.shape[0], batch_size):
            end = offset + batch_size
            sess.run(train_op, feed_dict={features: X_train[offset:end], labels: y_train[offset:end]})

        val_loss, val_acc = eval_on_data(X_val, y_val, sess)
        print("Epoch", i+1)
        print("Time: %.3f seconds" % (time.time() - t0))
        print("Validation Loss =", val_loss)
        print("Validation Accuracy =", val_acc)
        print("")

Extracting data:
(26270, 32, 32, 3)
(12939, 32, 32, 3)
(26270,)
(12939,)
Starting training...
Epoch 1
Time: 55.085 seconds
Validation Loss = 0.5083928263134894
Validation Accuracy = 0.8684596955403081

Epoch 2
Time: 54.277 seconds
Validation Loss = 0.3419855115301081
Validation Accuracy = 0.9158358451654723

Epoch 3
Time: 54.512 seconds
Validation Loss = 0.2555185901537331
Validation Accuracy = 0.9387124198390929

Epoch 4
Time: 54.430 seconds
Validation Loss = 0.23181654737338242
Validation Accuracy = 0.9400262771696439

Epoch 5
Time: 54.512 seconds
Validation Loss = 0.19534594838973388
Validation Accuracy = 0.9511554216166646

Epoch 6
Time: 54.535 seconds
Validation Loss = 0.1703036139517002
Validation Accuracy = 0.9562562795112487

Epoch 7
Time: 54.500 seconds
Validation Loss = 0.15102581787845507
Validation Accuracy = 0.9635211376689098

Epoch 8
Time: 54.481 seconds
Validation Loss = 0.14073186070222152
Validation Accuracy = 0.9644485663728282

Epoch 9
Time: 54.523 seconds
Validatio